In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from google.colab import drive, files
import io

# Cargar datos y dividir dataset

In [ ]:
from tensorflow.random import set_seed
set_seed(234730)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv("/content/drive/MyDrive/train.csv")
test = pd.read_csv('test_without_label.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train['id'] = train['id'].astype(str)
train['tweet'] = train['tweet'].astype(str)
train['label'] = train['label'].astype(str)

In [ ]:
train.drop(['id'], axis=1)
train.set_index("id", inplace = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# parseo de category
train[['label']] = train[['label']].apply(lambda col: label_encoder.fit_transform(col))

# Stopwords, lemmatize, punctuacion
Hay muchas palabras que estan repetidas como "august", "august,", "august.", "august:", etc. Para resolver esto vamos a sacar la punctuación antes de hacer el lemmatizer. Por ahora vamos a dejar algunos símbolos como los "#", quizas los hastags pueden tener información importante.

Como en el órden de ejecución del CountVectorizer primero se hace el lemmatizing,  no podemos ponerlo en las stop words. Por lo tanto lo vamos a hacer en el mismo método de lemmatizing.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
Lemmatizer = WordNetLemmatizer()
#analyzer = CountVectorizer().build_analyzer()

from zipfile import ZipFile
file_loc = '/root/nltk_data/corpora/wordnet.zip'
with ZipFile(file_loc, 'r') as z:
  z.extractall('/root/nltk_data/corpora/')

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#All_punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
All_punct = '''!()-[]{};:'"“”‘’\,<>./?%^&*_~'''
#CountVectorizer con stopwords de Natural Language Toolkit
nltk.download('stopwords')

def is_Not_Link(word):
  return word[0:4] != "http"

def remove_Punctuation(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    if is_Not_Link(word):
      for elements in word:
        if elements in All_punct:
          doc_split[i] = word.replace(elements, "")
  return doc_split

def remove_Stopwords(doc_split):
  doc_with_Stopwords = doc_split.copy();
  for i in range(len(doc_split)):
    word = doc_split[i];
    if word in nltk.corpus.stopwords.words('english'):
      doc_with_Stopwords.remove(word);

  return doc_with_Stopwords

def new_analyzer(doc):
  doc_split = doc.split();
  doc_split = remove_Punctuation(doc_split);
  doc_split = remove_Stopwords(doc_split);
  return (Lemmatizer.lemmatize(w.lower()) for w in doc_split);


nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Ingenieria de atributos

## Largo del documento
Uno de los atributos que nos interesa investigar es el largo de los atributos

In [ ]:
letters = "abcdefghijklmnñopqrstuvwxyz"
def contarLetras(row):
  count = 0;
  for letter in row['tweet']:
    if letter in letters:
      count = count +1;
  return count

def contarCovid(row):
  count = 0;
  if "covid" in row['tweet']:
    return 1
  return 0

def contarHashtags(row):
  count = 0;
  for letter in row['tweet']:
    if letter == "#":
      count = count +1;
  return count * 10

emojis = "ℹ️↗️⏺️▪️◾▶️☎️☑☺⚖️⛪✅✈✉️✔️❄️❌❌❗❤️➡➡️�🆕🆙🌍🌐🏠🏪🏥🏫👇👉👉🏾👨👩⚕️💉💊📃📈📉📍📏📢📨📺🔰🔴🔹️🕌🕐😷🙏🚶♀️🛡️🟢🤔🤱🏿🥼🧤🧪"
def contarEmojis(row):
  count = 0;
  for letter in row['tweet']:
    if letter in emojis:
      count = count + 1
  return count

In [ ]:
train['largoDocumento'] = train.tweet.str.len()
train['CantidadPalabras'] = train.apply(lambda row : len(row['tweet'].split()), axis=1)
train['cantidad de letras'] = train.apply(lambda row : contarLetras(row), axis=1)
train['cantidad de covid'] = train.apply(lambda row : contarLetras(row), axis=1)
train['cantidad de hashtag'] = train.apply(lambda row : contarHashtags(row), axis=1)
train['cantidad de emojis'] = train.apply(lambda row : contarEmojis(row), axis=1)

# X e Y
 Dividir los datos en X e Y

In [ ]:
from sklearn.model_selection import train_test_split
Y = train["label"]
X = train.drop('label', axis=1)


#X_train, X_validation, Y_train, Y_validation = train_test_split(
 #  train.drop('label', axis=1),
  #  train['label'],
   # test_size=(1.0/3), random_state=42)

## Fit

In [ ]:
TF_IDF_Vectorizer = TfidfVectorizer(analyzer=new_analyzer, max_features=15000)
tfidf_vector = TF_IDF_Vectorizer.fit(X["tweet"])

## Transform

In [ ]:
tfidf_vector = TF_IDF_Vectorizer.transform(X["tweet"])
tfidf_vector = pd.DataFrame(tfidf_vector.toarray(), index = X.index, columns = TF_IDF_Vectorizer.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Añadir atributos al nuevo dataframe de TFIDF

In [ ]:
tfidf_vector['largoDocumento'] = X['largoDocumento']
tfidf_vector['CantidadPalabras'] = X['CantidadPalabras']
tfidf_vector['cantidad de letras'] = X['cantidad de letras']
tfidf_vector['cantidad de covid']  = X['cantidad de covid'] 
tfidf_vector['cantidad de hashtag'] = X['cantidad de hashtag']
tfidf_vector['cantidad de emojis'] = X['cantidad de emojis']

# Red Neuronal

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LeakyReLU

In [ ]:
adam_optimizer='Adam'
loss='binary_crossentropy'
metric='accuracy'

model = Sequential()
model.add(Dense(20, input_shape=(tfidf_vector.shape[1],)))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dense(30, use_bias=True))
model.add(Activation('relu'))
model.add(Dense(1, use_bias=True))
model.add(Dropout(0.5))
model.add(Activation('sigmoid'))
model.compile(loss=loss, optimizer=adam_optimizer, metrics=[metric])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

X_train_array = np.asarray(tfidf_vector)
Y_train_array = np.asarray(Y)

training = model.fit(X_train_array, Y_train_array, epochs=200, batch_size=100, validation_split=0.2)

In [ ]:
import tensorflow.keras as keras

model.save("basic_model")
saved_model = keras.models.load_model("basic_model")

# Test

In [ ]:
test['id'].astype('str')
test['tweet'].astype('str')

test_bag = TF_IDF_Vectorizer.transform(test["tweet"])
test_bag = pd.DataFrame(test_bag.toarray(), index = test.index, columns = TF_IDF_Vectorizer.get_feature_names())

test['largoDocumento'] = test.tweet.str.len()
test['CantidadPalabras'] = test.apply(lambda row : len(row['tweet'].split()), axis=1)
test['cantidad de letras'] = test.apply(lambda row : contarLetras(row), axis=1)
test['cantidad de covid'] = test.apply(lambda row : contarLetras(row), axis=1)
test['cantidad de hashtag'] = test.apply(lambda row : contarHashtags(row), axis=1)
test['cantidad de emojis'] = test.apply(lambda row : contarEmojis(row), axis=1)

test_bag['largoDocumento'] = test['largoDocumento']
test_bag['largoDocumento'] = test['largoDocumento']
test_bag['CantidadPalabras'] = test['CantidadPalabras']
test_bag['cantidad de letras'] = test['cantidad de letras']
test_bag['cantidad de covid']  = test['cantidad de covid'] 
test_bag['cantidad de hashtag'] = test['cantidad de hashtag']
test_bag['cantidad de emojis'] = test['cantidad de emojis']


test_bag_array = np.asarray(test_bag)
test_prediction = saved_model.predict(test_bag_array)

copy = test_prediction.copy()
predNumber = [];
predLabel = [];
for i in range(len(copy)):
  if copy[i][0] > 0.5:
    predNumber.append([1])
    predLabel.append(["real"])
  else:
    predNumber.append([0])
    predLabel.append(["fake"])

prediction = pd.DataFrame(predLabel, columns=['label'])
prediction.index += 1
prediction = prediction.to_csv('prediction.csv')

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


67/67 [==============================] - 0s 1ms/step


In [ ]:
from google.colab import files

files.download('prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
k_fold_result_arr

[1.0, 1.0, 1.0, 1.0, 0.9349736379613357]